In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline

In [7]:
project_dir_path = '/n/data1/hms/dbmi/farhat/Roger/gatesMRI/Projects/Biomarkers'

#### Load Packages

In [8]:
import os
import pandas as pd
import numpy as np
from numpy import absolute
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import MaxNLocator
from matplotlib.gridspec import GridSpec
import matplotlib.pylab as pl
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import scipy.stats
from collections import Counter
import pickle
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error

#for exporting to Adobe Illustrator
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

Create directory for figures

In [9]:
figures_dir_path = f'{project_dir_path}/Figures/Notebook 10/'
if not os.path.exists(figures_dir_path):
    os.makedirs(figures_dir_path)

Specify directory paths

In [10]:
#directoy for Pickled Objects
pickled_objects_dir = f'{project_dir_path}/Data/Pickled Files/'

#specify directory for Data
data_dir = f'{project_dir_path}/Data/'

#RNAseq Data
RNAseq_annotation_file = f'{data_dir}Drug Treatment RNAseq Expression/CSV files/GSE40553/GSE40553_TreatMonitoring_MultivariateModel_input Info.csv'
RNAseq_gene_exp_matrix_file = f'{data_dir}Drug Treatment RNAseq Expression/CSV files/GSE40553/GSE40553_TreatMonitoring_MultivariateModel_input Exp_EachGene.csv'

Set parameters for plotting

In [11]:
plt.style.use('ggplot')
plt.rcParams['lines.linewidth']=1.0
plt.rcParams['axes.facecolor']='1.0'
plt.rcParams['xtick.color']='black'
plt.rcParams['axes.grid']=False
plt.rcParams['axes.edgecolor']='black'
plt.rcParams['grid.color']= '1.0'
plt.rcParams.update({'font.size': 14})

## [1] Load Data

### [1.1] Weighted Degrees for Nodes (genes) & Mean $log_2(Fold Change)$ for Nodes (genes)

#### ATB v HC

In [12]:
weighted_deg_ATB_HC_series = pd.read_pickle(pickled_objects_dir + 'Network Files/weighted degree series/ATB_v_HC.pkl')
mean_logFC_ATB_HC_series = pd.read_pickle(pickled_objects_dir + 'Network Files/mean logFC network nodes series/ATB_v_HC.pkl')

ATB_HC_df = pd.DataFrame(index = mean_logFC_ATB_HC_series.index)
ATB_HC_df['mean_log2FC'] = mean_logFC_ATB_HC_series.values
ATB_HC_df['weighted_degree'] = weighted_deg_ATB_HC_series[mean_logFC_ATB_HC_series.index].values

#### ATB v LTBI

In [13]:
weighted_deg_ATB_LTBI_series = pd.read_pickle(pickled_objects_dir + 'Network Files/weighted degree series/ATB_v_LTBI.pkl')
mean_logFC_ATB_LTBI_series = pd.read_pickle(pickled_objects_dir + 'Network Files/mean logFC network nodes series/ATB_v_LTBI.pkl')

ATB_LTBI_df = pd.DataFrame(index = mean_logFC_ATB_LTBI_series.index)
ATB_LTBI_df['mean_log2FC'] = mean_logFC_ATB_LTBI_series.values
ATB_LTBI_df['weighted_degree'] = weighted_deg_ATB_LTBI_series[mean_logFC_ATB_LTBI_series.index].values

#### ATB v OD

In [14]:
weighted_deg_ATB_OD_series = pd.read_pickle(pickled_objects_dir + 'Network Files/weighted degree series/ATB_v_OD.pkl')
mean_logFC_ATB_OD_series = pd.read_pickle(pickled_objects_dir + 'Network Files/mean logFC network nodes series/ATB_v_OD.pkl')

ATB_OD_df = pd.DataFrame(index = mean_logFC_ATB_OD_series.index)
ATB_OD_df['mean_log2FC'] = mean_logFC_ATB_OD_series.values
ATB_OD_df['weighted_degree'] = weighted_deg_ATB_OD_series[mean_logFC_ATB_OD_series.index].values

### [1.2] Top weighted nodes in (1) ATB/HC & ATB/LTBI & ATB/OD networks $(n = 24)$ and (2) ATB/HC & ATB/LTBI networks $(n = 23)$

In [15]:
with open(f'{pickled_objects_dir}Network Files/top weighted node lists/ATB_HC_and_ATB_LTBI_and_ATB_OD.pkl', 'rb') as f:
    top_genes_ATB_HC_and_ATB_LTBI_and_ATB_OD = pickle.load(f)

In [16]:
len(top_genes_ATB_HC_and_ATB_LTBI_and_ATB_OD)

24

In [17]:
with open(f'{pickled_objects_dir}Network Files/top weighted node lists/ATB_HC_and_ATB_LTBI.pkl', 'rb') as f:
    top_genes_ATB_HC_and_ATB_LTBI = pickle.load(f)

In [18]:
len(top_genes_ATB_HC_and_ATB_LTBI)

23

#### combine genes into a *single gene set*

In [19]:
gene_sig_set = top_genes_ATB_HC_and_ATB_LTBI_and_ATB_OD + top_genes_ATB_HC_and_ATB_LTBI

In [20]:
len(gene_sig_set)

47

### [1.3] Drug Treatment data from Wen-Han

In [21]:
gene_exp_info_df = pd.read_csv(RNAseq_annotation_file).set_index('Unnamed: 0')
gene_exp_info_df.index.rename('GSM_ID', inplace = True)

In [22]:
gene_exp_info_df.head()

,title,geographical location,status,time,disease,ID,group,groupName
GSM_ID,,,,,,,,
GSM996408,active TB pre-treatment [401/0],South Africa patient blood,active TB pre-treatment,0,PTB,401,1,0d prior to treatment
GSM996330,active TB 12 months post treatment [401/12],South Africa patient blood,active TB 12 months post treatment,360,PTB,401,7,360d after treatment
GSM996398,active TB pre-treatment [402/0],South Africa patient blood,active TB pre-treatment,0,PTB,402,1,0d prior to treatment
GSM996365,active TB 6 months post treatment [402/06],South Africa patient blood,active TB 6 months post treatment,180,PTB,402,6,180d after treatment
GSM996453,active TB 12 months post treatment [402/12],South Africa patient blood,active TB 12 months post treatment,360,PTB,402,7,360d after treatment


In [23]:
np.shape(gene_exp_info_df)

(131, 8)

In [24]:
gene_exp_matrix_df = pd.read_csv(RNAseq_gene_exp_matrix_file).set_index('Unnamed: 0')
gene_exp_matrix_df.index.rename('GSM_ID', inplace = True)
gene_exp_matrix_df = gene_exp_matrix_df.T

In [25]:
gene_exp_matrix_df.head()

GSM_ID,EEF1A1,GAPDH,LOC643334,SLC35E2,DUSP22,LOC642820,RPS28,IPO13,TESSP1,LOC653113,...,hematopoietic_progenitor,macrophage_m0,macrophage_m1,macrophage_m2,memory_B_cell,myeloid_dendritic_cell,naive_B_cell,neutrophil,plasma_cell,plasmacytoid_dendritic_cell
GSM996408,-0.262689,0.496376,0.306659,-0.281313,-0.882281,0.654244,-0.374078,-0.120265,-0.176062,0.205200,...,0.165424,0.031117,0.159177,0.000000,0.086135,0.0,0.000000,0.231802,0.096727,0.000000
GSM996330,-0.045556,0.015934,-0.369687,2.731641,1.562389,0.739739,-0.305610,0.278352,1.954948,0.342200,...,0.000000,0.000000,0.043323,0.000000,0.319639,0.0,0.000000,0.000000,0.021595,0.190413
GSM996398,0.153195,-0.086382,0.334537,1.816955,-0.854403,-0.548657,0.029551,0.455776,-0.148183,1.494945,...,0.000000,0.000000,0.284942,0.000000,0.388739,0.0,0.000000,0.000000,0.000000,0.000000
GSM996365,-0.416633,-0.761357,-0.654867,-1.242839,0.156193,-0.130404,0.271103,0.651323,-0.067198,-0.714906,...,0.000000,0.000000,0.053890,0.035400,0.000000,0.0,0.241711,0.173833,0.024274,0.000000
GSM996453,0.116076,-0.230482,-0.078740,-0.666712,0.732320,-0.694001,0.073566,-0.313820,1.640174,-0.544771,...,0.000000,0.000000,0.004827,0.005196,0.329905,0.0,0.046273,0.308311,0.063927,0.000000


In [26]:
np.shape(gene_exp_matrix_df)

(131, 31354)

## [2] Subset to top weighted Gene Sets for individuals during the duration of treatment

- **group 1** : 0 days (prior to treatment)

- **group 2** : 1 week after treatment

- **group 3** : 2 weeks after treatment

- **group 4** : 1 month after treatment

- **group 5** : 2 months after treatment

- **group 6** : 6 months after treatment

- **group 7** : Healthy Controls / 1 year after treatment

How many individuals are there in each group?

In [27]:
np.sum(gene_exp_info_df.group == 1)

28

In [28]:
np.sum(gene_exp_info_df.group == 2)

0

In [29]:
np.sum(gene_exp_info_df.group == 3)

25

In [30]:
np.sum(gene_exp_info_df.group == 4)

0

In [31]:
np.sum(gene_exp_info_df.group == 5)

24

In [32]:
np.sum(gene_exp_info_df.group == 6)

25

In [33]:
np.sum(gene_exp_info_df.group == 7)

29

- **group 1** : 0 days (prior to treatment) , **Y = 1**

- **group 3** : 2 weeks after treatment , **Y = 0.75**

- **group 5** : 2 months after treatment , **Y = 0.5**

- **group 6** : 6 months after treatment , **Y = 0.25**

- **group 7** : 1 year after treatment , **Y = 0**

In [34]:
group_ids = [1, 3, 5, 6, 7]
group_tags = ['prior\nto treatment', '2 weeks\nafter treatment', '2 months\nafter treatment', '6 months\nafter treatment', '1 year\nafter treatment']
Y_value_dict = {1:1 , 3:0.75 , 5:0.5 , 6:0.25 , 7:0}

### [2.1] subset to *N = 47* genes in gene signature & add *Y* values

In [35]:
exp_matrix_gene_set_df = gene_exp_matrix_df.loc[: , gene_sig_set]

In [36]:
np.shape(exp_matrix_gene_set_df)

(131, 47)

In [37]:
exp_matrix_gene_set_df

GSM_ID,GBP5,IFIT3,LAP3,VAMP5,PSTPIP2,C1QB,DUSP3,CEACAM1,WARS,XAF1,...,ID3,LY96,CD6,KLRB1,PIK3IP1,CCR7,FBXO6,KCNJ15,NELL2,SKAP1
GSM996408,2.544450,1.287279,2.371635,2.093394,2.147355,5.551621,1.430134,1.824227,2.428189,1.187352,...,-5.695193,1.249040,-1.303784,-1.401154,-1.461660,-1.758700,1.798810,1.751590,-2.582731,-0.962018
GSM996330,0.086348,-1.599552,-0.525319,0.003380,-0.432540,0.208641,-0.676584,-1.264306,-0.340603,-0.890235,...,0.422877,-1.109254,0.460270,0.120360,0.296327,0.085922,-0.241946,-2.270071,0.060675,0.528861
GSM996398,1.623428,1.970092,1.201545,0.372142,0.816581,2.024758,-0.001465,1.054112,1.031686,1.460158,...,0.838894,0.508862,0.215425,-0.509872,0.429892,0.836312,1.564942,1.068775,0.160567,-0.017245
GSM996365,-1.267674,-0.771007,-1.245346,-0.637052,0.255246,-3.293770,-0.476916,-0.052356,-0.546444,-0.152990,...,0.311372,-0.396123,-0.085485,-0.061872,0.321228,0.269428,-1.407828,0.313338,0.163415,0.059354
GSM996453,-0.628804,0.556417,-0.851327,-0.621060,0.137016,-2.717643,0.014793,0.533602,-0.147141,0.406824,...,0.257476,0.336761,0.320899,-0.412101,0.477473,0.794112,-0.675125,1.056227,0.586922,-0.046036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM996383,1.288971,1.507113,1.909472,1.464977,0.558301,4.374000,0.831762,-0.174944,0.477525,1.692915,...,-0.726658,0.881540,0.232200,-0.297763,-0.226818,-0.021709,1.627938,1.026735,-0.705438,0.066882
GSM996423,1.423941,0.574954,1.266092,0.338390,0.059441,3.587731,0.565715,-0.064677,0.538907,0.108438,...,-0.449672,0.188292,0.048969,-0.031226,-0.395335,0.054000,0.394088,0.517297,0.014982,-0.130964
GSM996331,0.969721,-0.566873,0.362199,0.465951,-0.143024,2.448046,-0.103071,-0.786555,0.604133,-0.172804,...,-0.110454,-0.526509,0.424464,0.272557,-0.214043,0.109027,0.390550,-0.825635,-0.042079,-0.017666
GSM996456,-0.167454,-2.031970,-0.758971,-0.392632,-0.001422,0.835281,-0.358298,-0.615335,0.042762,-0.658489,...,0.413656,-0.720811,0.475857,0.501611,0.448009,0.293797,-0.531339,-1.066135,0.564585,0.196114


In [38]:
print(exp_matrix_gene_set_df.mean().head(n=10))

GSM_ID
GBP5       0.522555
IFIT3      0.043764
LAP3       0.307040
VAMP5      0.191535
PSTPIP2    0.370002
C1QB       0.504029
DUSP3      0.304744
CEACAM1    0.189362
WARS       0.200622
XAF1       0.055311
dtype: float64


Variables (genes) don't look mean centered so we will **normalize** each variable when running **ElasticNet**

In [39]:
Y_values = [Y_value_dict[gene_exp_info_df.loc[subject_id , 'group']] for subject_id in exp_matrix_gene_set_df.index]
exp_matrix_gene_set_df.loc[: , 'Y'] = Y_values

In [40]:
exp_matrix_gene_set_df

GSM_ID,GBP5,IFIT3,LAP3,VAMP5,PSTPIP2,C1QB,DUSP3,CEACAM1,WARS,XAF1,...,LY96,CD6,KLRB1,PIK3IP1,CCR7,FBXO6,KCNJ15,NELL2,SKAP1,Y
GSM996408,2.544450,1.287279,2.371635,2.093394,2.147355,5.551621,1.430134,1.824227,2.428189,1.187352,...,1.249040,-1.303784,-1.401154,-1.461660,-1.758700,1.798810,1.751590,-2.582731,-0.962018,1.00
GSM996330,0.086348,-1.599552,-0.525319,0.003380,-0.432540,0.208641,-0.676584,-1.264306,-0.340603,-0.890235,...,-1.109254,0.460270,0.120360,0.296327,0.085922,-0.241946,-2.270071,0.060675,0.528861,0.00
GSM996398,1.623428,1.970092,1.201545,0.372142,0.816581,2.024758,-0.001465,1.054112,1.031686,1.460158,...,0.508862,0.215425,-0.509872,0.429892,0.836312,1.564942,1.068775,0.160567,-0.017245,1.00
GSM996365,-1.267674,-0.771007,-1.245346,-0.637052,0.255246,-3.293770,-0.476916,-0.052356,-0.546444,-0.152990,...,-0.396123,-0.085485,-0.061872,0.321228,0.269428,-1.407828,0.313338,0.163415,0.059354,0.25
GSM996453,-0.628804,0.556417,-0.851327,-0.621060,0.137016,-2.717643,0.014793,0.533602,-0.147141,0.406824,...,0.336761,0.320899,-0.412101,0.477473,0.794112,-0.675125,1.056227,0.586922,-0.046036,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM996383,1.288971,1.507113,1.909472,1.464977,0.558301,4.374000,0.831762,-0.174944,0.477525,1.692915,...,0.881540,0.232200,-0.297763,-0.226818,-0.021709,1.627938,1.026735,-0.705438,0.066882,1.00
GSM996423,1.423941,0.574954,1.266092,0.338390,0.059441,3.587731,0.565715,-0.064677,0.538907,0.108438,...,0.188292,0.048969,-0.031226,-0.395335,0.054000,0.394088,0.517297,0.014982,-0.130964,0.75
GSM996331,0.969721,-0.566873,0.362199,0.465951,-0.143024,2.448046,-0.103071,-0.786555,0.604133,-0.172804,...,-0.526509,0.424464,0.272557,-0.214043,0.109027,0.390550,-0.825635,-0.042079,-0.017666,0.50
GSM996456,-0.167454,-2.031970,-0.758971,-0.392632,-0.001422,0.835281,-0.358298,-0.615335,0.042762,-0.658489,...,-0.720811,0.475857,0.501611,0.448009,0.293797,-0.531339,-1.066135,0.564585,0.196114,0.25


## [3] Run Elastic Net

use code below to ignore covergence warnings

In [41]:
import warnings
warnings.filterwarnings('ignore')

Run Elastic Net on Data with gene expression for *47* genes as predictors

- Going to use repeated 10-fold crossvalidation on each combination of **ratio** and **alpha**

- **ratio**: float between 0 and 1 that is used for scaling between the **L1** and **L2** penalties 
    - ratio = 0, uses on L2 penalty and is equivalent to **Ridge Regression**
    - ratio = 1, uses on L1 penalty and is equivalent to **Lasso Regression**

- **alpha**: constant that multiplies the **L1** and **L2** penalties, tuning parameter that decides how much we want to penalize the model

- Want the combination of **ratio** and **alpha** that has the least **mean squared error**

In [42]:
#split the dataset DF into X & y
data = exp_matrix_gene_set_df.values
X, y = data[:, :-1], data[:, -1]

#define model evaluation method, 10-fold CV with 3 repeats (repeats K-Fold n times with diff. randomization in each repetition)
cv = RepeatedKFold(n_splits=10, n_repeats=3)

#set values for ratio & alpha that we will test
ratios = [.1, .5, .7, .9, .95, .99, 1]
alphas = [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1]

#define model, will cross validate over 
ElasticNet_model = ElasticNetCV(l1_ratio=ratios, alphas=alphas, cv=cv, normalize=True, n_jobs=-1)

#use ElasticNet linear model to fit model to data
ElasticNet_model.fit(X, y)

ElasticNetCV(alphas=[0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1],
             cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=None),
             l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], n_jobs=-1,
             normalize=True)

Check to see which values of **alpha** and **ratio** were chosen from crossvalidation

In [43]:
print('alpha: %f' % ElasticNet_model.alpha_)
print('l1_ratio_: %f' % ElasticNet_model.l1_ratio_)

alpha: 0.001000
l1_ratio_: 0.700000


print the **coefficients** for each gene/variable & check which were **non-zero**

In [44]:
ElasticNet_model.coef_

array([ 0.06278647,  0.        ,  0.0294829 , -0.02008184, -0.        ,
        0.        , -0.        , -0.        , -0.03414744,  0.        ,
        0.        ,  0.        ,  0.01534152,  0.        ,  0.        ,
        0.00445758, -0.00519993,  0.        ,  0.01922887,  0.        ,
        0.02063406,  0.09991415,  0.01264066, -0.        , -0.        ,
       -0.01540135, -0.        , -0.        ,  0.00304823,  0.        ,
        0.        ,  0.        ,  0.0640534 , -0.        ,  0.        ,
       -0.05702643, -0.        , -0.        ,  0.        ,  0.0326685 ,
       -0.12166396, -0.11994626,  0.02801572,  0.        ,  0.        ,
        0.        , -0.        ])

List of genes that had **non-zero** coefficients

In [45]:
gene_set_list = np.array(exp_matrix_gene_set_df.columns[:-1])
print(gene_set_list[ElasticNet_model.coef_ != 0])

['GBP5' 'LAP3' 'VAMP5' 'WARS' 'BATF2' 'SMARCD3' 'ADM' 'FCGR1B' 'EPSTI1'
 'GBP1' 'IFITM3' 'DYSF' 'LHFPL2' 'IL7R' 'TNFSF13B' 'CD6' 'KLRB1' 'PIK3IP1'
 'CCR7']


In [47]:
len(gene_set_list[ElasticNet_model.coef_ != 0])

19

What was the **mean squared error** for optimal **alpha** and **l1_ratio**?

In [50]:
optimal_l1_ratio = ElasticNet_model.l1_ratio_
optimal_alpha = ElasticNet_model.alpha_

In [49]:
np.shape(ElasticNet_model.mse_path_) #(n_l1_ratio , n_alpha , n_folds)

(7, 8, 30)

In [57]:
ElasticNet_model.mse_path_[ratios.index(optimal_l1_ratio), alphas.index(optimal_alpha)]

array([0.12568355, 0.13463334, 0.1449203 , 0.15119854, 0.17780762,
       0.13005664, 0.154516  , 0.11257541, 0.11605446, 0.10229605,
       0.14941872, 0.13868832, 0.14842939, 0.09669974, 0.14928741,
       0.15103695, 0.08720831, 0.10996785, 0.16960374, 0.16726412,
       0.10714286, 0.09431039, 0.17297886, 0.11958323, 0.15804477,
       0.18269231, 0.11354116, 0.20687717, 0.10088454, 0.12880258])

In [58]:
np.mean(ElasticNet_model.mse_path_[ratios.index(optimal_l1_ratio), alphas.index(optimal_alpha)])

0.13674014343391386